## 1. Введение

В предыдущем модуле мы рассмотрели базовые принципы работы с моделями на этапе продакшена.

МЫ НАУЧИЛИСЬ:

Img сохранять обученные модели в виде бинарных файлов, а также файлов других форматов, которые поддерживаются языками программирования, отличными от Python;

Img локально разворачивать собственный веб-сервис с помощью фреймворка Flask, встраивая в его работу свою модель машинного обучения;

Img оптимизировать работу этого веб-сервиса, увеличивая его пропускную способность с помощью таких инструментов, как uWSGI и NGINX.

Однако, как мы уже упомянули в конце прошлого модуля, осталась одна проблема: наш сервис (да и сама модель) работает только на нашем компьютере. Когда мы зальём исходный код на GitHub, а наш коллега Василий склонирует себе репозиторий и попробует запустить сервис на своём «боевом» сервере, нет гарантии, что приложение запустится.

Что, если на сервере Василия нет необходимых зависимостей, таких как Scikit Learn, Flask и так далее? Установить библиотеки несложно, но какие именно их версии нужны Василию? А что, если у него уже стоят некоторые библиотеки, которые вы использовали, но в совершенно другой версии, которая не совместима с вашим приложением?

Кроме того, для обеспечения работы сервиса в связке uWSGI + NGINX Василию придётся проделывать все те манипуляции, которые мы производили в прошлом модуле. Вы уже должны были убедиться, что процесс настройки взаимодействия uWSGI + NGINX + Flask не из лёгких, пусть его и можно обернуть в некоторую инструкцию.

А что, если, ко всему прочему, на сервере Василия стоит другая операционная система, например Windows, которая не поддерживает работу с uWSGI?

Так много вопросов… И у нас есть на них ответы. Проблема, к которой мы подошли, называется проблемой воспроизводимости. О ней, как правило, не задумываются новички, однако она часто проявляется на этапе продакшена. Например, мы хотим перенести проект с локальной машины, где вели разработку, на реальный сервер, но из-за разницы в настройках и конфигурациях этот процесс становится болезненным и требует затрат времени, которого всегда остаётся очень мало на этапе продакшена.

В этом модуле мы разберём, как сделать свой код воспроизводимым и как изолировать свой сервис так, чтобы его можно было запустить на любой машине, а самое главное — как сделать это быстро и просто.

ЦЕЛИ МОДУЛЯ:

Img Узнать, что такое воспроизводимость и какими инструментами её можно обеспечить.

Img Познакомиться с терминами «виртуализация» и «контейнеризация».

Img Научиться создавать виртуальные окружения, изолировать среду разработки, устанавливать и фиксировать зависимости в виртуальных окружениях.

Img Познакомиться с инструментом контейнеризации Docker.

Img Научиться писать Dockerfile, создавать образы контейнеров, запускать их, а также делиться ими с помощью хостинга docker-образов Docker Hub.

Img Создать docker-образ для нашего веб-сервиса и запустить его в контейнере.

С КАКИМ ПРОЕКТОМ БУДЕМ РАБОТАТЬ?

В предыдущем модуле мы написали маленький веб-сервис. В нём функционирует модель машинного обучения, которая выполняет предсказания для данных, поступающих через POST-запросы по эндпоинту '/predict'.

Перед прохождением модуля давайте зафиксируем, как должна выглядеть директория нашего проекта:

├─web
   ├─models
        └─model.pkl
   └─client.py
   └─server.py
Проект будет располагаться в директории web, в которой находятся файлы:

server.py — содержит интерфейс сервера, реализованный на Flask. В интерфейсе предусмотрено два эндпоинта:
'/' — корневой, по обращению к которому пользователю возвращается тестовое сообщение;
'/predict' — предназначенный для обработки POST-запросов. Запросы приходят в виде списка из четырёх чисел в формате json(). Результатом выполнения запроса является JSON-словарь с ключом 'prediction' и значением-предсказанием модели.
Вы писали функцию для обработки этого запроса в прошлом модуле.
client.py — скрипт для тестирования POST-запросов на сервер.
models — папка с моделями, в которой находится файл model.pkl с моделью.

## 2. Воспроизводимость

Код, разрабатываемый дата-сайентистом, должен быть воспроизводимым в постоянно меняющихся в условиях:

меняются данные, поступающие на вход;
трансформируются пайплайны предобработки данных;
постоянно изменяются гиперпараметры алгоритмов;
иногда модифицируются или вовсе удаляются алгоритмы популярных библиотек.
В идеале ожидается, что выполнение кода должно приводить к одинаковым результатам в различных условиях. Именно с этой целью в некоторых заданиях вы встречали требование зафиксировать параметр random seed — таким образом, вы уже неоднократно сталкивались с проблемой воспроизводимости на практике.

Воспроизводимость результатов является одним из главных показателей качества модели. Именно поэтому обеспечение воспроизводимости — одна из важнейших проблем в современном ML-инжиниринге.

Мы можем доверять результату, полученному с помощью модели, только когда доподлинно известно, на какой выборке и с каким пайплайном предобработки она была построена. Кроме того, ожидается, что результаты будут консистентными (согласованными с исходными данными), так как необходимо однозначно понимать, какое изменение привело к улучшению или ухудшению результатов.

Наиболее остро вопрос воспроизводимости стоит в случае, когда над одной моделью работает большая DS-команда. Каждый участник команды должен воспроизвести весь пайплайн и получить тот же самый результат. При этом возможные изменения должны доставляться в общий код так же быстро, сохраняя все требования к воспроизводимости.

На схеме приведён классический пайплайн работы над моделью:

ИНСТРУМЕНТЫ ОБЕСПЕЧЕНИЯ ВОСПРОИЗВОДИМОСТИ:

Версионирование кода.
Обычно весь пайплайн представлен в виде частей кода, для работы с которым используется знакомая нам распределённая система управления версиями Git в совокупности с хостингом GitHub.

Версионирование артефактов.
В процессе работы над проектом появляются различные артефакты: датасеты, модели, файлы конфигурации и прочее. Для их версионирования обычно используются такие инструменты, как DVC и Sonatype Nexus.

Виртуализация и контейнеризация.
Одним из важнейших аспектов воспроизводимости является настройка окружения.

Виртуализация — это технология изоляции, сохранения состояния и воспроизведения окружения. Благодаря ей вы можете воссоздать на другом компьютере точную копию вашего окружения и тем самым обеспечить воспроизводимость.

Однако иногда воссоздания окружения недостаточно и необходимо обеспечить не только воспроизводимость версий библиотек, но и воспроизводимость среды выполнения программы — операционной системы, на которой реализован проект. Для этого прибегают к инструментам контейнеризации, таким как Docker.

Все перечисленные выше инструменты мы подробно обсудим далее.

Управление экспериментами.
Для каждого запуска обучения необходимо фиксировать настройки гиперпараметров и сохранять их. В этом помогают системы управления экспериментами, например MLflow.

Над созданием универсального стандарта воспроизводимости работают многие крупные компании. Это означает, что у каждой компании пока что существуют собственные требования к обеспечению воспроизводимости. Когда вы начнёте работать в DS-команде, ваши коллеги будут требовать от вас их соблюдения. Обычно чем крупнее компания, тем более серьёзные требования к воспроизводимости необходимо соблюдать. Поскольку единого стандарта пока не существует, пути и используемые в разных компаниях инструменты могут кардинально отличаться.

Рекомендуем вам всегда помнить о задаче воспроизводимости результатов модели. Это позволит вам постепенно привить себе тот уровень культуры кода, который будет удовлетворять самые требовательные компании.

## 3. Виртуализация и изолированность. Virtualenv

При обсуждении воспроизводимости мы упоминали, что одним из важнейших её требований является настройка окружения. Разные версии библиотек в разных проектах могут конфликтовать друг с другом, и становится сложно поддерживать работоспособность всех сервисов. Поэтому необходимо обеспечить не только одинаковые операционные системы, но и установку всех нужных библиотек и их версий.

Почти в 100 % случаев при работе над реальным проектом вам потребуется воссоздать точное окружение на сервере, чтобы обеспечить единообразие среды выполнения. Это включает в себя не только стандартные установленные зависимости и интерпретатор Python, но и специфические зависимости, которые могут работать по-разному на разных операционных системах или на разных их версиях.

Несколько лет назад для этих задач использовали программные решения для удалённого управления конфигурациями. Они создавали большой конфигурационный файл, в котором описывались все настройки, зависимости и библиотеки, и на его основе настраивались все удалённые машины. Одним из таких инструментов является Ansible. Он позволяет через SSH-соединение «проталкивать» файлы конфигурации на множество машин и таким образом обеспечивать единообразие.

Другие системы, например Chef, обычно поступают наоборот: узлы «тянут» (pull) конфигурацию с главной машины. Chef используется и сейчас, когда необходимо привести парк машин к одинаковой конфигурации.

ПРОБЛЕМЫ

Когда в системе много сервисов и при этом каждый сервис запускается на разных машинах, становится тяжело следить за полнотой и правильностью конфигурационных файлов.
На локальном компьютере разработчика может быть не установлено (или наоборот установлено) ПО, напрямую влияющее на стабильность работы приложения.
Намного проще создать для каждого проекта виртуальную среду, изолированную от основной системы. Тогда каждый из проектов будет иметь свои независимые настройки, в том числе разные библиотеки и их версии.

Когда мы говорим про изолированность, мы имеем в виду, что приложение будет запускаться в своей отдельной среде и таким образом не зависеть от ОС и настроек системы, на которой мы его разворачиваем.

VIRTUALENV

Одним из наиболее популярных инструментов для создания изолированных сред в Python является virtualenv. Он обеспечивает работоспособность сервисов вне зависимости от того, какие они имеют зависимости.
Примечание. Сначала мы будем изучать основы работы с виртуальными окружениями на «игрушечных» примерах, а затем рассмотрим, как настроить виртуальное окружение для нашего веб-сервиса.

Предварительно создайте в своей операционной системе папку с именем project_a. Эта папка будет имитировать папку проекта, и на её основе мы продемонстрируем работу виртуальных окружений. Перейдите в созданную папку в терминале (команда cd /путь/до/папки) или откройте её в вашей IDE.

Чтобы установить virtualenv, необходимо воспользоваться стандартной командой:

$ pip install virtualenv

Примечание. Здесь и далее символ $ будет означать, что команды выполняются в терминале (для Windows — в командной строке). Вводить его не нужно — вводите только текст команды!

Напомним, что в VS Code терминал (в Windows — командную строку) можно открыть с помощью кнопки в верхнем меню IDE:

Так как принципы работы с инструментом незначительно, но отличаются для различных UNIX-систем (Linux и MacOS) и Windows, мы приведём инструкции по работе для каждой из ОС.

Примечание. Чтобы показать, что команды в терминале выполняются в виртуальном окружении, мы будем перед символом $ писать в скобках имя виртуального окружения, в котором происходит работа. Например:

(project_a_venv) $ pip install pandas

Чтобы создать новую среду, необходимо набрать в терминале команду:

$ python3 -m venv <название сервиса>
Например, следующая команда создаёт виртуальное окружение с именем project_a_venv:

$ python3 -m venv project_a_venv
После этого в вашей текущей директории появится папка проекта с именем, которое вы указали в команде venv.

В UNIX-системах эта директория будет выглядеть примерно следующим образом (имена файлов могут незначительно отличаться в зависимости от версии Python, которую вы используете):

img
В директории bin лежат файлы, которые взаимодействуют с виртуальной средой, а в lib и lib64 содержится копия версии Python и все зависимости (библиотеки и их версии).

Чтобы активировать виртуальную среду и зайти в нёе в UNIX-системах, необходимо запустить в терминале команду:

$ source project_a_venv/bin/activate
Если команда выполнилась успешно, вы увидите, что перед приглашением в командной строке появилась дополнительная надпись, совпадающая с именем виртуального окружения.

В результате выполнения команды мы увидим следующее:

img
Выделенная красным строка говорит нам, что мы находимся в изолированном окружении project_a_venv.

Таким образом вы полностью изолируете окружение своего проекта и можете установить все необходимые для работы проекта версии пакетов — эти версии не будут отражены в глобальном окружении Python и будут зафиксированы только в активированном виртуальном окружении.

Например, выполним команду для установки библиотеки scikit-learn:

(project_a_venv) $ pip install scikit-learn
После установки библиотеки вы увидите, что в папке lib/python3.Х/site-packages (X — ваша версия Python) появится папка scikit-learn, а также зависимости, необходимые для работы этой библиотеки (например, numpy, scipy и joblib) — они устанавливаются вместе с ней автоматически.

Чтобы выйти из виртуального окружения в область глобального окружения, необходимо ввести в терминале команду:

$ deactivate


VIRTUALENV И VS CODE

Если вы разрабатываете свои программы в IDE, например в VS Code, то перемещаться между виртуальными окружениями становится совсем просто.

Давайте предварительно создадим в папке нашего проекта пустой py-файл, чтобы VS Code понял, что мы работаем с языком Python. Назовём этот файл app.py.

Чтобы переключиться между окружениями в VS Code, необходимо перейти в раздел выбора интерпретатора Python (правый нижний угол):

img
По умолчанию используется глобальное окружение. Нам нужно переключиться на только что созданное виртуальное окружение:

img
После этого необходимо перезапустить терминал (если он был открыт).

Результат будет тем же, что и после активации виртуального окружения через командную строку:

ИЗОЛЯЦИЯ ЗАВИСИМОСТЕЙ

Теперь давайте на примере рассмотрим, как работать с виртуальными окружениями.

Рядом с папкой project_a создайте ещё одну папку проекта и назовите её project_b. В этой папке также создайте пустой файл app.py.

Откройте два терминала: в первом перейдите в папку project_a, а во втором — в project_b. Также можно открыть эти папки в двух окнах VS Code.

В папке с проектом А создадим виртуальное окружение с именем project_a_venv, активируем его и установим scikit-learn (если вы не делали этого ранее):

Примечание. Ключ -q предназначен для установки без вывода справочной информации — «тихая» установка (от англ. quiet — тихий).

Затем создадим виртуальное окружение в папке project_b с именем project_b_venv, активируем его и установим пакет pandas.

UNIXWINDOWS
$ python3 -m venv project_venv
$ source project_b_venv/bin/activate 
(project_b_venv)$ pip install -q pandas
Давайте посмотрим, какие библиотеки доступны внутри каждого из окружений. Для этого воспользуемся командой pip freeze, которая выводит список установленных пакетов с указанием номера их версии. Выполните в каждом из окружений команду:

(project_{}_venv)$ pip freeze
Для проекта А мы увидим примерно следующую картину:

img
Для проекта B список будет выглядеть так:

img
Примечание. На скриншотах приведены результаты работы команд только в UNIX-системах, так как они совпадают с результатами в Windows.

Что мы видим? Списки установленных пакетов отличаются: например, библиотеки scikit-learn нет в виртуальном окружении проекта B, а библиотеки pandas нет в виртуальном окружении проекта A. Списки пакетов для проектов А и B пересекаются только в одном — библиотеке numpy. Так происходит потому, что и scikit-learn, и pandas требуют для своей стандартной работы пакет numpy.

Теперь давайте взглянём на список глобально установленных пакетов. Для этого откроем ещё один терминал и, не активируя никаких окружений, напишем в нём команду для вывода списка установленных пакетов:

$ pip freeze
img
В глобальном окружении находится совершенно другой список зависимостей.

Примечание. Пакеты в глобальном окружении на вашем компьютере могут отличаться от приведённых на скриншоте.

Только что мы посмотрели на пример изоляции: мы создали отдельные виртуальные окружения для каждого из проектов, зависимости которых изолированы друг от друга. Из приведённого примера становится интуитивно понятно, что виртуальные окружения существуют независимо от глобального и установка пакета в одно из них не означает, что пакет будет установлен в какое-то другое окружение, и наоборот.

Например, если мы установим в глобальное окружение библиотеку numpy версии 1.19.2,

$ pip install numpy==1.19.2
то для виртуальных окружений проектов А и B версия numpy не изменится. Это утверждение справедливо и в обратную сторону. Такой механизм позволяет нам работать с проектами А и B независимо друг от друга и даже независимо от глобального окружения, тем самым гибко управляя проектами.

Примечание. Очевидно, что попытка запустить в окружении код, использующий библиотеку, которая в нём не установлена, приведёт к ошибке. Например, если попробовать импортировать библиотеку pandas в файле app.py проекта A,

Файл project_a/app.py

import pandas as pd

df = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})
print(df)
а после запустить этот файл из-под соответствующего виртуального окружения, мы получим ошибку импорта:

(project_a_venv) $ python3 app.py

Traceback (most recent call last):
  File "/home/andrey/prod-2/project_a/app.py", line 1, in <module>
    import pandas as pd
ModuleNotFoundError: No module named 'pandas'
Поэтому стоит иметь в виду, что для каждого создаваемого виртуального окружения необходимо отдельно установить все зависимости, которые не входят в стандартную библиотеку Python.
Однако это ещё не всё. Когда разработка проекта завершена и мы готовы загрузить его на свой GitHub и поделиться им с коллегами, мы можем сохранить все те версии библиотек, которые использовали при разработке, в файл. Для этого применяется всё та же команда pip freeze, только с указанием имени файла, в который необходимо произвести запись. Традиционно такой файл называют requirements.txt и располагают в корневой директории проекта. Для указания файла используется ключ -r или оператор >:

(project_a_venv) $ pip freeze -r requirements.txt
или

(project_a_venv) $ pip freeze > requirements.txt
В результате создаётся текстовый файл requirements.txt, который вы можете поместить в свой репозиторий на GitHub.

Когда коллеги будут клонировать ваш проект, им не нужно будет разбираться, какие библиотеки вы использовали в проекте, а тем более — каких версий. Они смогут создать на своём компьютере собственное виртуальное окружение и установить в него все необходимые зависимости, используя лишь одну команду:

(имя_виртуального_окружения) $ pip install requirements.txt
Удобно, не правда ли?

Теперь давайте заглянем «под капот» и поймём, как именно работает изоляция.



КАК РАБОТАЕТ ИЗОЛЯЦИЯ

Чтобы понять, как работает изоляция, давайте проверим, где располагаются исполняемые файлы Python в случае глобального и виртуального окружений.

Для начала посмотрим на глобальное окружение. С деактивированной средой запускаем команду:

Данная команда выводит расположение скрипта, который выполняется при вызове команды python3.

Теперь попробуйте запустить ту же самую команду, но уже находясь в одном из виртуальных окружений, например в виртуальном окружении проекта А:



Видно, что, активировав среду, мы получаем другой путь к интерпретатору Python. Но как подключение библиотек зависит от расположения интерпретатора? Напрямую.

Мы не будем подробно останавливаться на том, как происходит внутреннее управление путями в операционных системах, так как это довольно обширная тема и её изложение будет уникально для каждого типа операционных систем. Если говорить вкратце, при активации виртуального окружения вы меняете специальные переменные среды ОС, в результате чего при запуске команды вызова интерпретатора используете не глобальную версию интерпретатора Python, который имеет доступ ко всем установленным пакетам, а его копию, которая лежит в папке с виртуальным окружением. Этот интерпретатор не имеет доступа к папке, где хранятся библиотеки, установленные в глобальное или любое другое виртуальное окружение — у него она своя. Это же правило работает и в обратную сторону.

Примечание. Подробнее о том, как меняются переменные среды, можно узнать в этой статье.

ПРИЧИНЫ ИСПОЛЬЗОВАНИЯ ВИРТУАЛЬНЫХ ОКРУЖЕНИЙ

У вас мог возникнуть вопрос: зачем это нужно? Зачем так сложно? Раньше мы спокойно работали в глобальном окружении и даже не знали, что оно глобальное, а сейчас для каждого проекта придётся создавать отдельное виртуальное окружение?

Если вы хотите стать профессиональными разработчиками, работать в команде и обеспечивать воспроизводимость библиотек с минимальной затратой времени, то да — создавать виртуальные окружения вам придётся чуть ли не в каждом проекте.

Представим следующую ситуацию: у вас есть два проекта — проект A и проект B. Они оба имеют зависимость от одной и той же библиотеки. Проблема становится явной, когда мы начинаем запрашивать разные версии этой библиотеки. Например, может случиться так, что проект A запрашивает версию 1.0.0, а проект B — версию 2.0.0, причём версия 2.0.0 настолько сильно отличается от 1.0.0, что для адаптации проекта А под новую версию придётся его полностью переписывать. Это большая проблема для Python, ведь работая только в глобальном окружении, мы не можем использовать обе версии библиотеки. Так или иначе, возникнет конфликт, который могут решить виртуальные окружения.

Вторая причина использования виртуальных окружений — удобная коммуникация внутри команды. Разрабатывая проект в виртуальном окружении, мы можем сохранить только те зависимости и их версии, которые использовали в проекте, например в файл requirements.txt.

Затем мы можем передать разработку своим коллегами (например, через GitHub), и они смогут установить только те зависимости, которые необходимы для работы нашего кода.

Примечание. Важно отметить, что папку самого виртуального окружения в GitHub помещать не нужно. Всегда добавляйте эту папку в файл .gitignore.

Например, мы можем прописать в файле .gitignore строку *venv/, которая будет означать, что всё содержимое папок, названия которых оканчиваются на venv, будет игнорироваться при коммитах:



ВИРТУАЛЬНОЕ ОКРУЖЕНИЕ ДЛЯ FLASK-ПРИЛОЖЕНИЯ

Пришло время создать виртуальное окружения для нашего веб-сервиса.

Предварительно перейдите в каталог, где расположен код вашего проекта из предыдущего модуля.

В корневой директории проекта (у нас она называется web) создадим виртуальное окружение с именем project_venv.

$ python3 -m venv project_venv
Локальные копии Python и pip будут установлены в каталог project_venv в каталоге вашего проекта.

Активируем виртуальное окружение:

UNIXWINDOWS
$ source project_venv/bin/activate
Теперь мы можем переходить к установке пакетов в наше окружение. Сначала установим wheel с локальным экземпляром pip, чтобы убедиться, что наши пакеты будут устанавливаться даже при отсутствии архивов wheel:

(project_venv) $ pip install wheel
Затем установим Flask, requests и scikit-learn. Чтобы установить несколько пакетов сразу, можно просто перечислить их через пробел после команды install.

(project_venv) $ pip install flask requests scikit-learn
Затем запустим наш сервер:

UNIXWINDOWS
(project_venv) $ python3 ./server.py
Давайте проверим, что мы установили все необходимые для работы веб-сервиса и его тестирования зависимости. Через браузерную строку зайдите по адресу http://localhost:5000 или http://127.0.0.1:5000. Там должно быть выведено сообщение, что ваш сервер запущен.

Также попробуйте отправить POST-запрос на ваш сервер, выполнив клиентский скрипт в соседнем терминале.

UNIXWINDOWS
(project_venv) $ python3 ./client.py
В результате работы скрипта должно быть выведено сообщение о статусе обработки запроса (он должен быть равен 200) и предсказание модели для отправленных данных.

Если всё работает корректно, GET- и POST-запросы отработали без ошибок, то мы можем зафиксировать версии наших зависимостей и поместить их в файл requirements.txt в корневой директории проекта:

(project_venv) $ pip freeze > requirements.txt
В нашей директории появится файл requirements.txt — он ещё пригодится в следующих юнитах.

Теперь, если мы загрузим наш код на GitHub (предварительно добавив папку project_venv в файл .gitignore), нашему коллеге Василию необходимо будет склонировать себе репозиторий, а после этого создать виртуальное окружение и активировать его. Чтобы в точности воссоздать все версии зависимостей, которые мы использовали, Василию будет достаточно набрать в терминале следующую команду:

$ pip install requirements.txt
После этого зависимости внутри виртуального окружения, созданного Василием, будут совпадать с нашими.

Virtualenv — полезный инструмент. Однако не всё так гладко, ведь он работает только с Python и не обеспечивает полную изоляцию. Также он не позволяет ограничивать ресурсы для каждого сервиса: например, иногда бывает необходимо разрешить одному сервису использование всех ядер процессора и ограничить — другому. Если мы также хотим автоматически балансировать нагрузку между сервисами, то и тут virtualenv нам не помощник.

На эту тему есть прекрасная статья на Хабре.

Ещё один недостаток технологии виртуальных окружений состоит в том, что они не помогут, если разработка проекта ведётся на одной операционной системе, а эксплуатация — на другой. В частности, вы могли заметить, что при создании виртуального окружения для нашего веб-сервиса мы ничего не сказали о связке Flask, uWSGI и NGINX — как мы помним по прошлому модулю, эта связка поддерживается только для UNIX-систем. Поэтому нам необходимо создать не просто виртуальное окружение, где мы будет хранить необходимые для работы приложения библиотеки, а что-то вроде карманной операционной системы, внутри которой уже настроено взаимодействие всех необходимых инструментов для работы сервиса, включая установленные внутри версии библиотек. Причём хотелось бы, чтобы эту карманную ОС можно было запускать на любом устройстве, то есть чтобы наш сервис можно было переносить между платформами. Тут к нам на помощь приходят системы контейнеризации, в частности Docker.

## 4. Контейнеризация. Docker и Docker Hub

В 2012–2013 гг. появилось несколько систем контейнеризации, которые позволяли иметь дополнительную операционную систему в изолированном от основной ОС виде и не использовали много ресурсов.

Контейнеризация — это метод виртуализации, при котором ядро операционной системы поддерживает несколько изолированных экземпляров приложений.

Наиболее популярной системой контейнеризации оказался Docker. Сегодня, если кто-то говорит о контейнерах, скорее всего, имеется в виду именно Docker.

По сути, Docker — это программное обеспеспечение, которое позволяет в автоматическом режиме создавать виртуальные машины и управлять ими. При этом Docker делает это более элегантно по сравнению с обычными средствами виртуализации, например VirtualBox.

img
Что изображено на схеме выше?
Данные схемы демонстрируют отличия с точки зрения архитектуры информационной системы при использовании виртуальных машин и контейнеров. Схемы стоит читать снизу вверх.

В случае использования виртуальных машин (Virtual Machines) мы имеем:

Инфраструктуру системы (Infrastructure) — рабочие компьютеры пользователей, серверы, облачные технологии и т. д.
Операционную систему устройства (Host Operating System) — операционные системы (Windows/Unix/Mac), на которых работают компьютеры;
Гипервизор (Hypervisor) — процесс, которые отделяет операционную систему компьютера от физического оборудования. Проще говоря, это специальное приложение, которое позволяет создавать и управлять виртуальными машинами и запускать на одном компьютере множество различных виртуальных операционных систем. Например, для Windows таким приложением является Hyper-V. Подробнее о гипервизорах и их устройствах вы можете прочитать здесь.
Гостевые/виртуальные операционные системы (GuestOS) — операционные системы, которые работают внутри виртуальных машин. Их может быть сколько угодно (зависит от возможностей железа, установленного в компьютере, на котором запускаются виртуальные машины).
Бинарные файлы и библиотеки (Bins/Libs) — пакеты, которые необходимы для работы приложений, запущенных на каждой из гостевых ОС.
Приложения (App) — приложения, которые мы запускаем внутри гостевых ОС. Это может быть несколько серверных приложений, которые ожидают поступающих интернет-запросов, при этом каждое из них запущено под определённой ОС.
В случае использования контейнеров (Containers) мы имеем:

Инфраструктуру системы (Infrastructure).
Операционную систему устройства (Operating System).
Движок Докер (Docker Engine) — специальное программное обеспечение для автоматизации развёртывания и управления приложениями в средах с поддержкой контейнеризации. Проще говоря, это контейнеризатор приложений. Он позволяет «упаковать» приложение со всем его окружением и зависимостями в контейнер, который может быть развёрнут на любой операционной системе. В каждом контейнере запущены свои приложения (как правило, один контейнер — одно приложение) со своими зависимостями.
Бинарные файлы и библиотеки (Bins/Libs).
Приложения (App).
Таким образом, из схемы видно, что, в отличие от виртуальных машин, контейнерам не нужна установка отдельных гостевых операционных систем с ненужными функциями, такими как графический интерфейс, встроенные в ОС приложения и т. д. В каждом контейнере содержится своя микро-ОС, в которой можно изолированно запускать отдельные приложения.

Докер позволяет собрать приложение со всем его окружением и зависимостями в контейнер. В нашем случае приложением может быть модель ML, предсказывающая стоимость авто, его зависимостями — библиотеки sklearn, numpy и pandas, а окружением — ОС, на которой работает приложение.

За счёт того что Docker потребляет не очень много ресурсов машины, на которой находится, можно запускать сразу несколько контейнеров даже на среднестатистическом компьютере. Из-за этого стало принято использовать небольшие контейнеры для каждого конкретного сервиса: например, если у нас есть Django-приложение с базой данных, то сам сервер будет находиться в одном контейнере, а база — в другом. Изоляция часто позволяет добиться улучшения производительности и упрощения миграции сервисов.

img
Источник изображения

У Docker неслучайно такая эмблема и название!
Проще всего это представить, воспользовавшись метафорой кораблей и контейнеров. Если необходимо перевезти несколько типов грузов (продукты, тяжёлые машины и химикаты), то сделать это на одном корабле можно, только используя контейнеры и таким образом изолируя грузы друг от друга. Когда грузы находятся в контейнерах, уже не очень важно, что внутри — оно может быть погружено на корабль.

Таким образом, контейнеризация — это создание некоторого «ящика» для вашего приложения, в который будут сложены ядро, ОС, библиотеки, ПО и само приложение.

img
Docker работает с функциями системы Linux и её ядрами, поэтому при работе на других ОС он использует небольшую хитрость: каждый раз в систему устанавливается виртуальная машина с Linux, и Docker работает уже в ней, но мы этого даже не замечаем.

ПОДВЕДЁМ ПРОМЕЖУТОЧНЫЙ ИТОГ

Контейнер Docker, по сути, представляет собой «виртуальную» файловую систему, в которую вы устанавливаете всё необходимое для запуска вашего приложения. Это «всё необходимое» включает в себя даже ядро системы Linux. При запуске такого Docker ваша базовая система поднимает контейнер с этой файловой системой, и получается легковесная виртуальная машина, что-то вроде карманной операционной системы.

Образы (images) — это основные строительные блоки, на основании которых создаются контейнеры (а в них впоследствии упаковываются приложения).

Образ, по своей сути — это шаблон, своеобразный чертёж или рецепт, в котором содержится образ базовой операционной системы, код приложения и библиотеки.

Запуская на его основе контейнер, мы создаём исполняемый экземпляр, который инкапсулирует требуемое программное обеспечение.

Ключевую роль в устройстве образа играет идея о слоях.

img
В основе каждого docker лежит базовый образ с операционной системой. С каждым новым слоем в ОС добавляются другие компоненты. Каждый слой представляет из себя подобие diff файловой системы. Например, вы взяли образ системы Ubuntu и поставили туда Python. В таком случае ваш образ будет состоять из двух слоёв — самой ОС и файлов Python поверх неё.

В качестве базового образа для docker можно использовать не только ОС, но и готовые образы с нужными вам компонентами. Кто угодно может добавлять свои образы в общий регистр, поэтому в нём очень много готовых образов, доступных для расширения. Таким образом, можно взять готовый образ из публичного репозитория в качестве базового и добавлять в него дополнительные слои. Это делается в соответствии с инструкциями из Dockerfile, которые мы подробно рассмотрим дальше.

Готовые образы хранятся в Docker Registry. Они могут быть публичными или приватными: например, официальное публичное хранилище — это Docker Hub. Это аналог GitHub, но если последний используется для хранения кода и работы с Git, то Docker Hub используется для хранения docker-образов и работы с Docker.

В качестве базового можно использовать абсолютно любой образ. Однако стоит помнить о безопасности и удостовериться, что образ не навредит.

img
Источник изображения
Основное преимущество Docker заключается в том, что с помощью Docker Registry мы можем делиться образами, а значит, легко переносить созданные нами приложения.

Для работы с Docker Registry достаточно знать две команды (они очень похожи на команды Git) — подробнее о них мы поговорим далее:

docker push — отправить собранный образ в Docker Registry.
docker pull — скачать готовый образ из Docker Registry.
Существует множество официальных образов. Например, можно найти образ, где в качестве ОС используется Ubuntu, или, например, Linux с уже установленным Python. Есть даже Docker в Docker! Очень удобно, не правда ли?

Docker-образ управляется Daemon, который отвечает за все действия, связанные с контейнерами, и, конечно, самим клиентом для взаимодействия с ним.

Важно. Зарегистрируйтесь на Docker Hub. Для этого вам понадобится только e-mail. Ваш аккаунт пригодится вам далее при выполнении заданий, а также при работе с самим Docker.

УСТАНОВКА DOCKER

Существует две версии Docker: Docker Community Edition (CE) и Docker Business. Версия Community содержит бесплатный набор продуктов Docker. Корпоративная (Business) версия является сертифицированной и представляет собой контейнерную платформу, предоставляюшую своим пользователям дополнительные платные функции, например управление образами, безопасность образов, оркестрирование и управление средой выполнения контейнеров.

Для наших базовых целей будет достаточно и Community-версии, но имейте в виду, что большие компании, для которых важны безопасность и сертификация, а также гибкость работы с внутренней частью контейнеров, как правило, работают в Business-версии.

Исконно Docker предназначался только для операционных систем Linux. Контейнеры Docker, созданные в конкретной операционной системе, используют ядро ОС. Иначе говоря, это означает, что мы не можем использовать ядро Windows для запуска контейнеров Linux или наоборот.

Однако для систем Windows и MacOS есть обходной путь — приложение Docker Desktop. Поэтому, прежде чем начинать работу с Docker, пользователям этих ОС необходимо установить это приложение.

Docker Desktop — это декстопное приложение от разработчиков Docker, первоначально предназначенное для пользователей Windows и MacOS. С помощью графического интерфейса этого приложения можно создавать, тестировать и публиковать свои приложения, предварительно завёрнутые в Docker-контейнеры.

img

Также вместе с Docker Desktop поставляются сервисы, которые понадобятся нам в дальнейшем, в частности:

Движок Docker (Docker Engine) — включает в себя инструменты для построения контейнеров, реестр контейнеров, инструменты оркестрации, среду выполнения и многое другое.
Docker Compose — инструмент, который предназначен для организации взаимодействия нескольких контейнеров Docker. Мы поговорим о нём в следующем модуле.
Ещё несколько компонентов, которые для нас сейчас не очень важны, но при желании вы можете подробнее узнать о них здесь.
Хитрость Docker Desktop заключается в том, что для своих контейнеров он запускает Docker на виртуальной машине под операционной системой Linux. То есть, по сути, при сборке контейнеров вы будете использовать ядро ОС Linux, но даже не заметите этого. Благодаря этой особенности ваши контейнеры можно будет запускать на других компьютерах с ОС Linux или ОС, где установлен Docker Desktop.

Мы будем учиться работать с контейнерами, используя инструменты командной строки (терминала) и не прибегая к графическому интерфейсу Docker Desktop. Однако если вы изучите основные концепции работы с консольным Docker, для вас не составит труда разобраться и в графическом интерфейсе этого приложения.

Итак, давайте перейдём к установке. Все её шаги уже описаны в инструкциях с официального сайта — мы лишь приведём их здесь и обратим ваше внимание на некоторые нюансы.

Примечание. Если вы являетесь пользователем Windows, но работаете с дистрибутивами Linux через WSL, то для вас актуальна установка Docker Desktop для Windows с последующей интеграцией Docker в дистрибутив Linux через WSL. Более подробную информацию о работе с Docker через Windows WSL вы можете найти здесь.

## 5. Создание docker-образов. Dockerfile


Нам уже пора учиться создавать собственные образы, запускать контейнеры и управлять ими. В этом юните мы рассмотрим основной синтаксис работы с Docker на примере приложения для визуализации данных, а уже в следующем юните применим технологию контейнеров для нашего веб-сервиса и заставим его работать на любой машине.



Dockerfile — это специальный файл, в котором содержатся инструкции по сборке контейнера (а точнее, его слоёв): какой тип контейнера и операционную систему использовать, какие дополнительные пакеты установить, какие команды запустить.

Так, например, если нам нужен образ для решения задач машинного обучения, то в Dockerfile мы пропишем инструкцию включить в образ библиотеки sklearn или tensorflow.

Каждая такая команда приводит к созданию отдельного слоя: diff слоя при добавлении файлов будет состоять из файлов, при выполнении команды — из разницы в файловой системе до выполнения и после.

Типичный Dockerfile выглядит примерно так:

img
Давайте на практике разберёмся, что всё это значит, и соберём свой образ контейнера.

Внимание! В видео эксперт работает в IDE PyCharm. Вы можете работать в той IDE, к которой привыкли.

Примечание. В видео для построения графиков плотности в файле plot.py эксперт применяет функцию distplot() из библиотеки Seaborn. Эта функция будет удалена в версии 0.14.0, и её не рекомендуется использовать. По этой причине в коде, приведённом в тексте модуля, мы заменили функцию distplot() на функцию histplot() для построения гистограмм с параметром kde=True (отображение плотности вероятности).

Также мы обновили версии библиотек до более актуальных (подробнее об этом — ниже, в разделе «Добавляем библиотеки в контейнер»).

Кроме того, в целях сохранения дальнейшей воспроизводимости мы изменили базовый образ на python:3.9 (в видео эксперт использует python:latest).
Мы хотим написать сервис, который создаёт две случайные подвыборки данных из нормальных распределений: первая выборка — с параметрами  и  (параметры стандартного нормального распределения), вторая — с параметрами, которые ввёл пользователь.

Для каждой выборки должны строиться и сохраняться в файл plot.png графики плотности распределений. Все файлы с графиками будем помещать в папку output.

Мы хотим запускать это приложение в контейнере и обеспечивать его работу на любом компьютере.

После окончания работы наш проект будет содержать:

само приложение plot.py;
папку output, в которой будут храниться результаты работы;
приложения, то есть графики плотности распределений в формате PNG;
Dockerfile — описание контейнера, чтобы обеспечить работу приложения на любом компьютере;
файл requirements.txt — зависимости приложения (библиотеки, которые мы используем).
Примечание. Для простоты изложения в данном проекте не используется виртуальное окружение, но мы уверены, что вы самостоятельно можете его создать и установить в него все необходимые зависимости (numpy, matplotlib, seaborn).

ШАГ 1. ПИШЕМ ПРИЛОЖЕНИЕ

Начнём с написания самого приложения. Пусть сначала исходный код в файле plot.py и папка output будут находиться в корневой директории my_first_container:

ШАГ 2. СОЗДАЁМ DOCKER-ОБРАЗ

Наше мини-приложение работает. Теперь мы можем завернуть его в контейнер, поэтому следующим шагом будет создание образа контейнера. Для этого создайте (если вы этого ещё не сделали) файл Dockerfile (без расширения) в корне папки, в которой лежит ваше приложение.

Традиционно код самого приложения помещают в папку src (от англ. source — источник), а Dockerfile и файл с зависимостями requirements.txt (о последнем мы поговорим далее) располагают рядом с этой папкой в корневой директории проекта. Тогда директория нашего проекта будет выглядеть следующим образом:

my_first_container
      ├─src
          └─output
          └─plot.py
      └─Dockerfile
Откройте Dockerfile в любом текстовом редакторе или в привычной IDE.

Переходим к описанию нашего контейнера. Первое, что нужно обязательно указать в нашем новом Dockerfile, — какой образ мы берём за основу. Базовый образ будет задавать файловую систему контейнера и многие другие его составные конфигурации.

Давайте загрузим в качестве основы образ Linux с уже установленным Python. Для указания базового образа, на основе которого будет собираться контейнер, используется ключевое слово FROM. Итак, вот первая строка нашего Dockerfile:

FROM python:3.9
Примечание. Здесь :3.9 в названии базового образа указывает на его версию. В данном случае мы используем версию 3.9 — с ней в нашем коде не возникнет предупреждений. Но вы можете использовать другие версии: например, python:latest означает использование последней доступной версии.

Примечание. В описании образа можно ознакомиться с информацией о том, какие бывают сборки и версии. Например, существуют обычная сборка python:<version> и сборка python:<version>-alpine, созданная на базе Alpine Linux. Последняя весит намного меньше, чем большинство базовых образов дистрибутива (~ 5 МБ). Кроме того, есть сборка python:<version>-slim, которая содержит только минимальные пакеты, необходимые для запуска Python, и не включает в себя стандартные пакеты.

Теперь укажем путь к рабочей папке нашего приложения внутри docker-контейнера (вместо /usr/src/app вы можете прописать любой путь, по которому хотите поместить файлы внутри контейнера). Для этого используется ключевое слово WORKDIR.

WORKDIR /usr/src/app
Примечание. Важно понимать, что файловая система контейнера существует отдельно от вашей. Так как образ python, который мы используем в качестве базового, собран на основе Linux, то и файловая система контейнера будет, как в ОС Linux. Ключевое слово WORKDIR говорит контейнеру, какой каталог будет использоваться при его запуске. В данном случае мы говорим, что та директория, с которой по умолчанию будет запускаться контейнер, — это /usr/src/app.

Далее скопируем в рабочую директорию файлы из корня нашего приложения. Для этого используется ключевое слово COPY. В качестве первого аргумента этой директивы указываются папки или файлы, которые мы хотим скопировать с локальной машины, в качестве второго аргумента — куда мы хотим поместить эти копии в контейнере.

COPY ./src/ ./
Что здесь происходит? Мы указываем, чтобы при создании образа содержимое папки ./src/, в которой находится исходный код нашего приложения, было перемещено в рабочую папку (корневой каталог ./) . В результате при сборке образа контейнера в рабочей директории образа появятся файл plot.py и папка output.

img
Наконец, напишем команду для запуска скрипта, который будет выполняться вместе с запуском контейнера. Для этого используется директива CMD (от англ. command — команда):

CMD [ "python", "./plot.py" ]
Таким образом, промежуточный Dockerfile будет выглядеть так:

Файл Dockerfile

FROM python:3.9
WORKDIR /usr/src/app
COPY ./src/ .
CMD [ "python", "./plot.py" ]
Теперь откроем терминал, перейдём в папку с нашим приложением и запустим команду для сборки контейнера (docker build):

$ docker build -t my_first_image .
Разберём команду на составляющие:

build сообщает docker, что мы хотим создать образ;
ключ -t указывает на название образа;
. в конце означает, что Dockerfile нужно искать именно в корне. Так как мы запускаем команду из директории my_first_containter/, а в ней находится Dockerfile, то Docker автоматически найдёт его. Если ваш Dockerfile находится в директории, отличной от той, в которой вы запускаете команду docker build, то вместо "." вам необходимо будет указать путь до него.
Запускаем команду и видим, что образ успешно создан. В справочной информации отражены все наши сборки. Если на каком-то этапе произойдёт ошибка при сборке, docker уведомит вас об этом в терминале.

img
Чтобы убедиться, что образ собран, а также посмотреть список доступных образов, можно использовать команду:

$ docker images
Если вы ранее не работали с Docker, скорее всего, сейчас вы получите следующие образы: my_first_image, python (он использовался в качестве базового для my_first_image) и hello-world (мы запускали его на этапе установки Docker).

Команда docker images выводит список собранных образов в виде таблицы со следующими столбцами: REPOSITORY (имя образа), TAG (тег образа, в котором обычно указывается его версия), IMAGE ID (идентификатор образа, по которому его можно однозначно найти, — у вас он может отличаться), CREATED (как давно образ был собран), SIZE (размер образа).

После выполнения команды docker images мы увидим на экране примерно следующую таблицу:

img

Из неё видно, что образу my_first_image соответствует тег latest.

Примечание. Так как образы занимают место в памяти компьютера, неактуальные образы можно удалять с помощью команды docker rmi (от англ. remove image):

$ docker rmi <image_id>
где image_id — идентификатор образа (столбец IMAGE_ID).

ШАГ 3. ЗАПУСКАЕМ DOCKER-КОНТЕЙНЕР

Попробуем запустить контейнер на основе нашего образа — для этого используется команда docker run <имя образа>:

$ docker run -it --rm --name=my_first_container my_first_image
Расшифруем ключи и аргументы этой команды:

-it объединяет команды: -i оставляет строку для ввода, а -t выделяет терминал;
параметр --rm автоматически удаляет контейнер после завершения его работы (в том числе при завершении с ошибкой) — это позволяет не хранить неактивные контейнеры;
параметр --name назначает docker-контейнеру имя (мы задали имя my_first_container).
Но вот незадача — после запуска мы увидим ошибку Python:

Traceback (most recent call last):
  File "./plot.py", line 4, in <module>
    import seaborn as sns
ModuleNotFoundError: No module named 'seaborn'
Эта ошибка говорит нам, что какие-то библиотеки не установлены. Почему это произошло? Ответ — в самом определении Docker: это инструмент виртуализации. То есть окружение Docker полностью изолировано от нашей операционной системы. Более того, внутри нашего образа и вовсе используется другая операционная система, а это значит, что внутри контейнера нет тех библиотек, которые есть на нашем компьютере.

В таком случае давайте добавим нужные зависимости в Dockerfile.

ШАГ 4. ДОБАВЛЯЕМ БИБЛИОТЕКИ В КОНТЕЙНЕР

Для этого создадим файл requirements.txt рядом с Dockerfile. Укажем в нём необходимые библиотеки и их версии:

my_first_container
      ├─src
          └─output
          └─plot.py
      └─Dockerfile
      └─requirements.txt
Файл requirements.txt

numpy == 1.23.4;
matplotlib == 3.6.0;
seaborn == 0.11.2.
Теперь необходимо переместить файл с зависимостями в наш контейнер, а после запустить команду для установки зависимостей.

Для этого напишем перед командой CMD, запускающей выполнение скрипта в Dockerfile, строки:

COPY ./requirements.txt ./
RUN pip install --no-cache-dir -r requirements.txt
Директива COPY нам уже знакома: она позволяет копировать файлы из локальной директории в файловую систему контейнера.

img
Директива RUN позволяет запускать любые команды по аналогии с терминалом. Например, поскольку мы работаем на основе базового образа Python, мы можем воспользоваться менеджером пакетов pip для установки зависимостей.

Также для команды pip install мы указываем:

параметр --no-cache-dir — позволяет не использовать кэш, а скачать пакеты заново. Вы можете не указывать этот параметр, если уверены, что в дальнейшем версии используемых библиотек не изменятся.
ключ -r — указывает на файлы с зависимостями.
Тогда наш итоговый Dockerfile, на основе которого будет собираться образ контейнера, будет выглядеть следующим образом:

Файл ./Dockerfile

FROM python:3.9
WORKDIR /usr/src/app
COPY ./src/ ./
COPY ./requirements.txt ./
RUN pip install --no-cache-dir -r requirements.txt
CMD [ "python", "./plot.py" ]
Теперь нам необходимо заново собрать образ (сборка займёт некоторое время):

$ docker build -t my_first_image .
После этого можно запустить контейнер на основе этого образа:

$ docker run -it --rm --name=my_first_container my_first_image
Примечание. Для того чтобы посмотреть список запущенных контейнеров, можно воспользоваться командой docker ps. По умолчанию она выводит список активных контейнеров. Для вывода списка всех контейнеров используйте ключ -a, но предварительно перезапустите контейнер без ключа --rm, так как данный ключ удаляет контейнеры.

$ docker ps -a
Запустите команду в соседнем терминале одновременно с контейнером.

Она выводит на экран информацию о контейнерах в виде таблицы со следующими столбцами:

CONTAINER ID — идентификатор контейнера;
IMAGE — имя образа, на основе которого запущен контейнер;
COMMAND — команда, используемая внутри контейнера (то, что мы прописали в директиве CMD в Dockerfile);
CREATED — когда был запущен контейнер;
STATUS — статус контейнера;
PORTS — порты, которые использует контейнер (о них поговорим в следующем юните — сейчас наше приложение не использует веб-интерфейс, и портов у него не будет);
NAMES — имя контейнера.
Теперь всё работает корректно. После запуска контейнера вас попросят ввести среднее и стандартное отклонение. После исполнения контейнера на экран должна быть выведена фраза "Файл успешно сохранен".

Однако файл plot.png в папке output не обновился с выполнением скрипта в контейнере. Куда же он тогда «успешно сохранён»?

ШАГ 5. СИНХРОНИЗАЦИЯ ПУТЕЙ, ИЛИ КУДА ПРОПАЛ PLOT.PNG

Технически файл plot.png удалился вместе с контейнером после того, как запустился и выполнился скрипт plot.py. Docker не сохраняет файлы внутри контейнера, так как внутри контейнера своя отдельная файловая система.

Что же делать? Ответ очень прост — нам нужно связать контейнер с локальной файловой системой на нашем компьютере.

Для этого укажем параметр --volume или ключ -v в команде docker run.

Ключ -v требует указания путей, которые записываются в формате <путь на локальной машине>:<путь в контейнере>:

$ docker run -it --rm -v $PWD/src/output/:/usr/src/app/output  --name=my_first_container my_first_image
Если всё сделано правильно, после выполнения команды в папке output появится нужный график.

ШАГ 6. ЗАГРУЖАЕМ ОБРАЗ НА DOCKER HUB

Большой успех Docker во многом обусловлен возможностью делиться образами контейнеров в Docker Hub.

Принцип загрузки образа в Docker Hub очень схож с загрузкой кода на GitHub:

создаем локальный образ контейнера;
делаем push образа на Docker Hub.
Пользователь со своей стороны может:

сделать pull нашего образа;
запустить образ контейнера на своей локальной машине.
Если мы правильно собрали образ контейнера, то благодаря технологии виртуализации пользователю не нужно знать, как устроено наше приложение, на какой операционной системе оно работает, какие зависимости в нём установлены — ему нужно просто запустить наш образ на своей на локальной машине. Это значительно упрощает разработку внутри команды и выведение конечных решений в продакшн.

Рассмотрим механизм загрузки образов в Docker Hub по шагам.

Первым делом зарегистрируйтесь на Docker Hub, если не сделали этого ранее.
Далее необходимо залогиниться под своей учётной записью в самом Docker. Для этого наберите в терминале команду:

$ docker login
Если вы ранее залогинились в Docker Desktop, данные учётной записи подгрузятся автоматически. Если вы не устанавливали Docker Desktop и не логинились в нём, Docker попросит вас ввести данные учётной записи на Docker Hub.

При отправке образа в Docker Hub необходимо указать имя пользователя как часть имени образа, так как Docker Hub организует репозитории по имени пользователя. Любой репозиторий, созданный под учётной записью, включает имя пользователя в имя образа Docker.
Поэтому нам необходимо пересобрать наш образ my_first_image, задав ему имя в формате <username>/server_name, где <username> — это ваше имя пользователя в профиле на Docker Hub.

$ docker build -t <username>/my_first_image .
Далее делаем push нашего образа на Docker Hub. В аргументах команды необходимо указать имя образа, которым мы хотим поделиться:

$ docker push <username>/my_first_image
Далее откройте ваш профиль на Docker Hub — в нём должен был появиться новый репозиторий с именем <username>/my_first_image:

img
Вы можете зайти внутрь репозитория и отредактировать его по своему усмотрению. Для этого используется опция Manage Repository:

img
В открывшемся окне вы увидите страницу своего docker-репозитория. Здесь вновь всё очень похоже на GitHub: есть раздел с кратким описанием образа (Description), а также файл README.md.

img
Примечание. При желании в файле README.md вы можете на языке MarkDown описать суть образа, а также команды, которые необходимы для его запуска — всё как с GitHub-репозиториями.

После того как образ выложен на Docker Hub, любой пользователь может скачать его и воспользоваться им. Для этого используется команда docker pull, в аргументах которой указывается имя репозитория на Docker Hub:

$ docker pull <username>/my_first_image
После этого образ контейнера станет доступным для запуска (это можно отследить с помощью команды docker images), и его можно запустить стандартной командой run:

$ docker run -it --rm -v $PWD/src/output/:/usr/src/app/output  --name=my_first_container my_first_image
Обратите внимание: для запуска образов, которые мы получаем через pull, не нужно их собирать, то есть нам не нужен Dockerfile (данный файл по определению предназначен для сборки контейнера). Контейнер уже собран, и нам достаточно запустить его.

НА ЭТОМ ВСЁ?

Иногда нам может потребоваться задать переменные среды, которые используются для определения констант. Чтобы задать их, в Dockerfile существует инструкция ENV — благодаря ей значение будет доступно в контейнере во время его выполнения.

Так в Linux-подобных системах называются переменные, которые определяются на уровне оболочки и используются различными приложениями во время выполнения. Подробнее о них можно узнать в статье.
Синтаксис ENV можно найти в официальной документации.

В Dockerfile нужно добавить:

ENV <key> <value>
или

ENV <key>=<value> ...
Или, к примеру, при запуске контейнера:

$ docker run --env <key>=<value>
Например, мы хотим, чтобы в нашем контейнере появилась переменная среды NAME, в которой будет указываться имя разработчика контейнера. К этой переменной среды можно будет обратиться из любой части контейнера. Тогда в Dockerfile необходимо добавить объявление такой переменной среды:

Файл ./Dockerfile

FROM python:3.9
ENV NAME="Skillfactory" 
WORKDIR /usr/src/app
COPY ./src/ ./
COPY ./requirements.txt ./
RUN pip install --no-cache-dir -r requirements.txt
CMD [ "python", "./plot.py" ]
ПРОМЕЖУТОЧНЫЙ ВЫВОД ПО РАБОТЕ С DOCKER

В этом юните мы на примере маленького приложения рассмотрели, как создавать собственные образы, а также основные директивы Dockerfile. Запоминать последние не обязательно — вы всегда можете найти их в документации или в этом гайде. Также мы изучили основные команды, необходимые для работы c Docker.

Перечислим их (и пару дополнительных) ещё раз:

docker build — создать образ контейнера;
docker images — посмотреть список доступных образов;
docker rmi — удалить образ;
docker run — запустить контейнер;
docker stop — остановить контейнер;
docker rm — удалить контейнер;
docker ps — посмотреть список запущенных контейнеров;
docker login - залогиниться в учётной записи Docker Hub;
docker push - отправить образ на Docker Hub;
docker pull - скачать образ с Docker Hub;
docker logs — посмотреть логи;
docker kill — экстренно завершить процесс.
Полезно! У Docker достаточно богатый CLI (Command-Line Interface), который содержит множество команд, помимо тех, что перечислены в модуле. Больше команд и информации о них вы можете увидеть в официальной документации.

## 6. Создаём образ веб-сервиса


Теперь давайте завернём в контейнер веб-сервис для нашего коллеги Василия, а затем загрузим его на Docker Hub, чтобы этот сервис был доступен для использования и Василий смог развернуть его у себя.

ШАГ 1. ЗАВОРАЧИВАЕМ FLASK-ПРИЛОЖЕНИЕ В КОНТЕЙНЕР

Прежде всего, давайте ещё раз договоримся о расположении файлов в нашей директории. Вынесем файлы с кодом сервера (server.py и папка models) и клиентским приложением (client.py) в папки app и test соответственно. Это нужно для удобства, чтобы не прописывать несколько команд COPY в Dockerfile.

In [1]:
!pip install clyent==1.2.1
!pip install nbformat==5.4.0
!pip install requests==2.28.1

  Preparing metadata (setup.py) ... done
  Created wheel for clyent: filename=clyent-1.2.1-py3-none-any.whl size=9177 sha256=1997aa34d8fa91b0f165848d44881caba34c94c9cabc1d86c08e721bf9f1f0e9
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/f8/18/64/668f0be15646b951e5f692bdab1c61cab099b486cae1bc09b8
Successfully built clyent
  Attempting uninstall: clyent
    Found existing installation: clyent 1.2.2
    Uninstalling clyent-1.2.2:
      Successfully uninstalled clyent-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.41 requires requests==2.28.1, but you have requests 2.29.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: nbformat
    Found existing inst